In [ ]:
'''
<Transfer Learning with Fine-tuning>
In this script, we implement a simple transfer learning example with fine-tuning for the task of classifying
cats and dogs using a trained model of VGG19 on ImageNet dataset. 
We remove the last FC layers of the VGG19 and attach a simple top model classifier. 
We have also chosen to fine tune the last block (block5) of VGG16 model and our top model which would increase the accuracy.
'''

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model

# set parameters here
# path to top model weights
top_model_weights_path = '\\bottleneck_fc_model_vgg19.h5'
# image size
img_width, img_height = 150, 150
# path to train and validation data, note that each folder includes subfolders of subtypes, e.g. cat and dog
train_data_dir = '\\data\\all\\train'
validation_data_dir = '\\data\\all\\validation'
# total number of images for training
nb_train_samples = 2000
# total number of images for validation
nb_validation_samples = 800

epochs = 50
batch_size = 16

# load VGG19 network with ImageNet weights as base_model
base_model = applications.VGG19(weights='imagenet', include_top=False, input_shape = (img_width, img_height, 3))
print('VGG19 model is loaded ...')

# build a top classifier model and attach to the VGG19 model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
print('Top model is created ...')


# load top model trained weightes
top_model.load_weights(top_model_weights_path)
print('Top model weights loaded ...')

# add the model on top of the convolutional base
# model.add(top_model)
model = Model(input= base_model.input, output= top_model(base_model.output))

# set the first 17 layers of VGG19 to non-trainable
for layer in model.layers[:17]:
    layer.trainable = False

# compile the model
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model
history = model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)


In [ ]:
# save model and weights
model.save_weights('model_weights_TransferLearning_FineTuning_EasyCatDog_vgg19.h5')

# Save the model architecture
with open('model_architecture_TransferLearning_FineTuning_EasyCatDog_vgg19.json', 'w') as f:
    f.write(model.to_json())

In [ ]:
# load back model and weights from saved files
from keras.models import model_from_json

# Model reconstruction from JSON file
with open('model_architecture_TransferLearning_FineTuning_EasyCatDog_vgg19.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('model_weights_TransferLearning_FineTuning_EasyCatDog_vgg19.h5')

In [ ]:
# save model block-wise illustration as tiff image
from keras.utils import plot_model
model.summary()
plot_model(model, to_file='model_vgg19_topmodel.tiff', show_shapes=True)

In [ ]:
# plot accuracya nd loss plots for train and val data
import matplotlib.pyplot as plt
print(history.history.keys())
# accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()
# loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='center right')
plt.show()